In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import lingualTF as la

import tensorflow as tf
#import keras

from keras.models import Sequential
from keras.layers import Convolution1D, AveragePooling1D, AveragePooling1D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization

[nltk_data] Downloading package wordnet to /Users/Seth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


Using TensorFlow backend.


In [ ]:
def DSIacc(y_true, y_pred):
    
    

## get data from file (processed in earlier scripts)

In [9]:
####################################
# REPLACE THESE WITH CORRECT PATHS #
####################################
docRanks = pd.read_csv('/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/refData/docRanks.csv')
#docRanks.head()
docRanks = docRanks.ix[~((docRanks['groupName']=='YV03')|(docRanks['groupName']=='YV04'))]
docRanks.reset_index(inplace=True, drop=True)
docRanks.tail()

,groupName,rank
273,WBC347,2
274,WBC410,1
275,WBC418,1
276,WBC421,1
277,WBC422,1


In [10]:
# OR load X matrix from disk (already processed)
X = np.load('/Users/Seth/Documents/DSI/Capstone/big-data/X-single-1k-278.npy')
X.shape

(278, 500, 1002)

In [11]:
# construct Y matrix
Y = np.array([np.array([y]) for y in docRanks['rank']])
Y.shape

(278, 1)

In [18]:
# make a Y vector for classification
Y_cat = np.zeros((len(Y),3))

for i in range(0,len(Y)):
    if Y[i] < 3:
        Y_cat[i][0] = 1
    elif Y[i] > 5:
        Y_cat[i][2] = 1
    else:
        Y_cat[i][1] = 1

Y_cat.shape

(278, 3)

## shuffle the data into training and testing

In [22]:
# set the seed if you want to
np.random.seed(123)
#
shuf = docRanks.sample(frac=1).index
shuf

Int64Index([223,  20, 163,  29, 254, 237,  53, 174, 151, 156,
            ...
             96, 225, 214,  57, 123, 106,  83,  17, 230,  98],
           dtype='int64', length=278)

In [23]:
splitPoint = int(len(X) * .8)
#train = docRanks.iloc[shuf[:splitPoint]]
#test = docRanks.iloc[shuf[(splitPoint+1):]]
splitPoint

222

In [24]:
X_train = X[shuf[:splitPoint]]
Y_train = Y[shuf[:splitPoint]]
Y_cat_train = Y_cat[shuf[:splitPoint]]

X_test = X[shuf[(splitPoint+1):]]
Y_test = Y[shuf[(splitPoint+1):]]
Y_cat_test = Y_cat[shuf[(splitPoint+1):]]

## classification model - no dropout

In [30]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [31]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [32]:
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [34]:
#model.fit(data, labels, nb_epoch=10, batch_size=32) ### generic call from documentation
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)
#model.fit(X_train, Y_cat_train, nb_epoch=10, batch_size= 1)

Epoch 1/10
222/222 [==============================] - 15s - loss: 1.4876 - acc: 0.4955    
Epoch 2/10
222/222 [==============================] - 11s - loss: 0.8438 - acc: 0.6757    
Epoch 3/10
222/222 [==============================] - 11s - loss: 0.3217 - acc: 0.8874    
Epoch 4/10
222/222 [==============================] - 11s - loss: 0.3264 - acc: 0.9189    
Epoch 5/10
222/222 [==============================] - 11s - loss: 0.1609 - acc: 0.9550    
Epoch 6/10
222/222 [==============================] - 11s - loss: 0.0856 - acc: 0.9685    
Epoch 7/10
222/222 [==============================] - 11s - loss: 0.1787 - acc: 0.9459    
Epoch 8/10
222/222 [==============================] - 11s - loss: 0.6190 - acc: 0.7883    
Epoch 9/10
222/222 [==============================] - 13s - loss: 0.1376 - acc: 0.9595    
Epoch 10/10
222/222 [==============================] - 15s - loss: 0.1654 - acc: 0.9685    


### evaluate

In [35]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

55/55 [==============================] - 2s     


[1.0854847417636351, 0.59999999999999998]

In [36]:
y_pred = model.predict(X_test)
#y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred[:10]

[[2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]

In [37]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [38]:
DSIacc(Y_test,y_pred_class)

0.41818181818181815

# DROPOUT

## MSE 1 batch

In [39]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [40]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.5))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.25))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [41]:
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)

In [42]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [43]:
#model.fit(data, labels, nb_epoch=10, batch_size=32) ### generic call from documentation
#model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)
model.fit(X_train, Y_cat_train, nb_epoch=10, batch_size= 1)

Epoch 1/10
222/222 [==============================] - 54s - loss: 0.2174 - mean_absolute_error: 0.4393 - acc: 0.5721    
Epoch 2/10
222/222 [==============================] - 49s - loss: 0.2090 - mean_absolute_error: 0.4293 - acc: 0.5856    
Epoch 3/10
222/222 [==============================] - 49s - loss: 0.2028 - mean_absolute_error: 0.4208 - acc: 0.5856    
Epoch 4/10
222/222 [==============================] - 54s - loss: 0.1986 - mean_absolute_error: 0.4127 - acc: 0.5856    
Epoch 5/10
222/222 [==============================] - 54s - loss: 0.1957 - mean_absolute_error: 0.4072 - acc: 0.5856    
Epoch 6/10
222/222 [==============================] - 60s - loss: 0.1937 - mean_absolute_error: 0.4023 - acc: 0.5856    
Epoch 7/10
222/222 [==============================] - 60s - loss: 0.1923 - mean_absolute_error: 0.3981 - acc: 0.5856    
Epoch 8/10
222/222 [==============================] - 57s - loss: 0.1915 - mean_absolute_error: 0.3944 - acc: 0.5856    
Epoch 9/10
222/222 [============

### evaluate

In [44]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

55/55 [==============================] - 2s     


[0.50909092426300051, 0.50909092426300051, 0.23636363636363636]

In [49]:
y_pred = model.predict(X_test)
y_pred[:10]

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]], dtype=float32)

In [46]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [47]:
DSIacc(Y_test,y_pred_class)

0.509090909090909

## MSE 32 batch

In [50]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [51]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [52]:
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)

Epoch 1/10
222/222 [==============================] - 11s - loss: 0.2536 - mean_absolute_error: 0.4427 - acc: 0.3243    
Epoch 2/10
222/222 [==============================] - 8s - loss: 0.2159 - mean_absolute_error: 0.4065 - acc: 0.4414     
Epoch 3/10
222/222 [==============================] - 8s - loss: 0.2260 - mean_absolute_error: 0.4099 - acc: 0.4324     
Epoch 4/10
222/222 [==============================] - 9s - loss: 0.2087 - mean_absolute_error: 0.3913 - acc: 0.4865     
Epoch 5/10
222/222 [==============================] - 9s - loss: 0.2072 - mean_absolute_error: 0.3880 - acc: 0.4820     
Epoch 6/10
222/222 [==============================] - 8s - loss: 0.1957 - mean_absolute_error: 0.3753 - acc: 0.5360     
Epoch 7/10
222/222 [==============================] - 7s - loss: 0.1952 - mean_absolute_error: 0.3720 - acc: 0.5315     
Epoch 8/10
222/222 [==============================] - 7s - loss: 0.1852 - mean_absolute_error: 0.3590 - acc: 0.5541     
Epoch 9/10
222/222 [============

#### evaluate

In [53]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

53/55 [===========================>..] - ETA: 0s

[0.20503065613183108, 0.42477386973120951, 0.59999999999999998]

In [58]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]

In [56]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

## Crossentropy 1 batch

In [59]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [60]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [61]:
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=1)

Epoch 1/10
222/222 [==============================] - 49s - loss: 1.0779 - acc: 0.5811    
Epoch 2/10
222/222 [==============================] - 46s - loss: 1.0438 - acc: 0.5856    
Epoch 3/10
222/222 [==============================] - 47s - loss: 1.0183 - acc: 0.5856    
Epoch 4/10
222/222 [==============================] - 49s - loss: 1.0001 - acc: 0.5856    
Epoch 5/10
222/222 [==============================] - 44s - loss: 0.9872 - acc: 0.5856    
Epoch 6/10
222/222 [==============================] - 44s - loss: 0.9777 - acc: 0.5856    
Epoch 7/10
222/222 [==============================] - 44s - loss: 0.9713 - acc: 0.5856    
Epoch 8/10
222/222 [==============================] - 44s - loss: 0.9666 - acc: 0.5856    
Epoch 9/10
222/222 [==============================] - 45s - loss: 0.9638 - acc: 0.5856    
Epoch 10/10
222/222 [==============================] - 44s - loss: 0.9618 - acc: 0.5856    


#### evaluate

In [62]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[12.601420064405961, 0.21818181818181817]

In [63]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[1], [3], [1], [1], [3], [1], [3], [1], [1], [3]]

In [64]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

## Crossentropy 32 batch

In [65]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [66]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [67]:
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)

Epoch 1/10
222/222 [==============================] - 9s - loss: 1.3654 - acc: 0.4324     
Epoch 2/10
222/222 [==============================] - 8s - loss: 0.9720 - acc: 0.6351     
Epoch 3/10
222/222 [==============================] - 7s - loss: 0.5855 - acc: 0.7658     
Epoch 4/10
222/222 [==============================] - 7s - loss: 0.3746 - acc: 0.8739     
Epoch 5/10
222/222 [==============================] - 7s - loss: 0.2940 - acc: 0.8874     
Epoch 6/10
222/222 [==============================] - 7s - loss: 0.3209 - acc: 0.8919     
Epoch 7/10
222/222 [==============================] - 7s - loss: 0.2277 - acc: 0.9099     
Epoch 8/10
222/222 [==============================] - 7s - loss: 0.1139 - acc: 0.9640     
Epoch 9/10
222/222 [==============================] - 7s - loss: 0.0601 - acc: 0.9820     
Epoch 10/10
222/222 [==============================] - 7s - loss: 0.0348 - acc: 0.9955     


#### evaluate

In [68]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[1.1995809855786237, 0.59999999999999998]

In [69]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]

In [70]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

## ...running for more epochs (back to batch size 1)

In [71]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(3, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [72]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [73]:
model.fit(X_train, Y_cat_train, nb_epoch=20, batch_size= 32)

Epoch 1/20
222/222 [==============================] - 9s - loss: 1.2198 - acc: 0.4775     
Epoch 2/20
222/222 [==============================] - 7s - loss: 1.0029 - acc: 0.5721     
Epoch 3/20
222/222 [==============================] - 7s - loss: 0.6373 - acc: 0.7568     
Epoch 4/20
222/222 [==============================] - 7s - loss: 0.4591 - acc: 0.8468     
Epoch 5/20
222/222 [==============================] - 7s - loss: 0.4265 - acc: 0.8333     
Epoch 6/20
222/222 [==============================] - 7s - loss: 0.1856 - acc: 0.9189     
Epoch 7/20
222/222 [==============================] - 7s - loss: 0.1512 - acc: 0.9505     
Epoch 8/20
222/222 [==============================] - 7s - loss: 0.1477 - acc: 0.9550     
Epoch 9/20
222/222 [==============================] - 7s - loss: 0.1562 - acc: 0.9459     
Epoch 10/20
222/222 [==============================] - 7s - loss: 0.0859 - acc: 0.9685     
Epoch 11/20
222/222 [==============================] - 7s - loss: 0.0854 - acc: 0.9685   

### evaluate

In [74]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[4.053578041315169, 0.59999999999999998]

In [75]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]

In [76]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [145]:
DSIacc(Y_test,y_pred_class)

0.18421052631578946